In [1]:
import logging
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
documents = ['Human machine interface for lab abc computer applications',
            'A survey of user opinion of computer system response time',
             'The EPS user interface management system',
             'System and human system engineering testing of EPS',
             'Relation of user perceived response time to error measurement',
             'The generation of random binary unordered trees',
             'The interaction graph of paths in trees',
             'Graph minors IV Widths of trees and well quasi ordering',
             'Graph minors A survey'
            ]

Tokenize the documents, remove common words (using a toy stoplist) as well as words that only appear once in the corpus

In [4]:
from pprint import pprint
from collections import defaultdict

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())

texts = [[word for word in document.lower().split() if word not in stoplist]
        for document in documents
        ]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [5]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
print(dictionary)

E:\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2019-09-19 16:49:44,148 INFO : 'pattern' package not found; tag filters are not available for English
2019-09-19 16:49:44,237 INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-09-19 16:49:44,238 INFO : built Dictionary(12 unique tokens: ['user', 'minors', 'system', 'eps', 'response']...) from 9 documents (total 29 corpus positions)


Dictionary(12 unique tokens: ['user', 'minors', 'system', 'eps', 'response']...)


In [6]:
print(dictionary.token2id)

{'user': 7, 'minors': 11, 'system': 5, 'eps': 8, 'response': 3, 'time': 6, 'survey': 4, 'graph': 10, 'trees': 9, 'human': 1, 'computer': 0, 'interface': 2}


In [7]:
# to actually convert tokenized documents to vectors
new_doc = 'Human computer interaction'
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(0, 1), (1, 1)]


In [8]:
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus)

[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


In [9]:
class MyCorpus(object):
    def __iter__(self):
        for line in open('mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

In [10]:
corpus_memory_friendly = MyCorpus()
print(corpus_memory_friendly)

In [11]:
for vector in corpus_memory_friendly:
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [14]:
from six import iteritems
# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('mycorpus.txt'))
# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
# remove stop words and words that appear only once
dictionary.filter_tokens(stop_ids + once_ids)
# remove gaps in id sequence after words that were removed
dictionary.compactify()
print(dictionary)

2019-09-19 17:07:26,292 INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-09-19 17:07:26,293 INFO : built Dictionary(42 unique tokens: ['management', 'user', 'opinion', 'system', 'eps']...) from 9 documents (total 69 corpus positions)


Dictionary(12 unique tokens: ['user', 'time', 'system', 'eps', 'response']...)
